In [1]:
import pamtra2
import collections
import refractiveIndex
import numpy as np
import pandas as pn
import xarray as xr
from collections import OrderedDict
from copy import deepcopy, copy
import toolz
import collections
import matplotlib.pyplot as plt

try:
    %matplotlib inline
except:
    pass

# create Profile

In [2]:
additionalDims = collections.OrderedDict()
additionalDims['time'] = pn.date_range('2016-01-01','2016-01-05',freq='D')[:1]
additionalDims['lat'] = np.arange(70,80)
nHeights = 100

pam2 = pamtra2.pamtra2(
    nLayer=nHeights,
    hydrometeors=['rain','snow'],
    additionalDims = additionalDims,
    frequencies = [35e9, 94e9],
)



In [3]:
pam2.profile

<xarray.customProfile>
Dimensions:              (frequency: 2, hydrometeor: 2, lat: 10, layer: 100, time: 1)
Coordinates:
  * time                 (time) datetime64[ns] 2016-01-01
  * lat                  (lat) int64 70 71 72 73 74 75 76 77 78 79
  * layer                (layer) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...
  * hydrometeor          (hydrometeor) <U4 'rain' 'snow'
  * frequency            (frequency) float64 3.5e+10 9.4e+10
Data variables:
    height               (time, lat, layer) float64 nan nan nan nan nan nan ...
    temperature          (time, lat, layer) float64 nan nan nan nan nan nan ...
    pressure             (time, lat, layer) float64 nan nan nan nan nan nan ...
    relativeHumidity     (time, lat, layer) float64 nan nan nan nan nan nan ...
    horizontalWind       (time, lat, layer) float64 nan nan nan nan nan nan ...
    verticalWind         (time, lat, layer) float64 nan nan nan nan nan nan ...
    eddyDissipationRate  (time, lat, layer) float64 nan na

Fill with some dummie values

In [4]:

pam2.profile.height[:] = np.linspace(0,1000,nHeights)
pam2.profile.temperature[:] = 250 
pam2.profile.relativeHumidity[:] = 90
pam2.profile.pressure[:] = 100000
pam2.profile.eddyDissipationRate[:] = 1e-4
pam2.profile.horizontalWind[:] = 0



pam2.profile.waterContent.values[:] = 0
#rain
pam2.profile.waterContent.values[...,20:40,0] = 1e-4
#snow
pam2.profile.waterContent.values[...,20:40,1] = 2e-4



For testing, dask arrays are turned on by chunking.

In [5]:
pam2.profile = pam2.profile.chunk({'time':1, 'hydrometeor':1, 'lat':1, 'frequency':1})

In [6]:
pam2.addMissingVariables()
pam2.profile 

<xarray.customProfile>
Dimensions:              (frequency: 2, hydrometeor: 2, lat: 10, layer: 100, time: 1)
Coordinates:
  * time                 (time) datetime64[ns] 2016-01-01
  * lat                  (lat) int64 70 71 72 73 74 75 76 77 78 79
  * layer                (layer) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...
  * hydrometeor          (hydrometeor) <U4 'rain' 'snow'
  * frequency            (frequency) float64 3.5e+10 9.4e+10
Data variables:
    height               (time, lat, layer) float64 dask.array<shape=(1, 10, 100), chunksize=(1, 1, 100)>
    temperature          (time, lat, layer) float64 dask.array<shape=(1, 10, 100), chunksize=(1, 1, 100)>
    pressure             (time, lat, layer) float64 dask.array<shape=(1, 10, 100), chunksize=(1, 1, 100)>
    relativeHumidity     (time, lat, layer) float64 dask.array<shape=(1, 10, 100), chunksize=(1, 1, 100)>
    horizontalWind       (time, lat, layer) float64 dask.array<shape=(1, 10, 100), chunksize=(1, 1, 100)>
    vert

In [7]:
# # OR EASIER if desired: 
# pam2 = pamtra2.importers.profiles.usStandardAtmosphere(heigths)

# Describe Hydrometeors
To describe the hydrometeors, you can use functions, xr.DataArrays or scalars. Teh functions must support xr.DataArrays. The kwargs of the functions are gathered from (in this order)

1. pam2.hydrometors.name.profile variables (name must match). E.g. density (after it has been calculated) 
2. pam2.profile variables (name must match). E.g. temperature
3. Other provided kwargs (e.g. Dmin, DMax, N0 etc)
4. Default function arguments. Can be turned of with useFuncArgDefaults = False to make sure everything is explicitly covered.



In [8]:

pam2.describeHydrometeor(
    pamtra2.hydrometeors.softEllipsoidFixedDensity,
    name = 'rain', #or None, then str(index)
    nBins =40,
    sizeBounds = pamtra2.hydrometeors.size.linspaceBounds, 
    sizeCenter = pamtra2.hydrometeors.size.boundsToMid,
    sizeBoundsWidth = pamtra2.hydrometeors.size.boundsWidth,
    sizeDistribution = pamtra2.hydrometeors.sizeDistribution.exponentialN0WC, 
    aspectRatio = 1.0,
    mass = pamtra2.hydrometeors.mass.ellipsoid,
    density = pamtra2.hydrometeors.density.water,
    crossSectionArea = pamtra2.hydrometeors.crossSectionArea.sphere,
    relativePermittivity = refractiveIndex.water.turner_kneifel_cadeddu, #replace with refractiveIndex.water.Turner.n
    scattering = pamtra2.hydrometeors.scattering.Rayleigh,
    fallVelocity = pamtra2.hydrometeors.fallVelocity.khvorostyanov01_drops,
    Dmin  = 1e-6,
    Dmax  = 1e-2,
    N0 =  8e6,
    useFuncArgDefaults = True,
)


sizeBounds <function linspaceBounds at 0x11ab85488>
callable
sizeCenter <function boundsToMid at 0x11ab85598>
callable
sizeBoundsWidth <function boundsWidth at 0x11ab85620>
callable
aspectRatio 1.0
not callable 1.0
density 1000.0
not callable 1000.0
mass <function ellipsoid at 0x31bb65730>
callable
crossSectionArea <function sphere at 0x31bb65d90>
callable
sizeDistribution <function exponentialN0WC at 0x31bb65ae8>
callable
relativePermittivity <function turner_kneifel_cadeddu at 0x41c767bf8>
callable


/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2/hydrometeors/sizeDistribution.py:301: UserWarning: Truncation effect on the PSD are not considered. I.e., typically mass is lost!
  warnings.warn('Truncation effect on the PSD are not considered. '
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/dask/local.py:271: RuntimeWarning: divide by zero encountered in true_divide
  return func(*args2)


scattering <function Rayleigh at 0x31bb6e268>
callable
fallVelocity <function khvorostyanov01_drops at 0x41c7a0950>
callable
nBins 40
not callable 40
Dmin 1e-06
not callable 1e-06
Dmax 0.01
not callable 0.01
N0 8000000.0
not callable 8000000.0


/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/_collections_abc.py:720: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  yield from self._mapping


Note that there is a warning in case not all arguments are used

In [9]:
pam2.describeHydrometeor(
    pamtra2.hydrometeors.softEllipsoidMassSize,
    name='snow',
    nBins = 20,
    sizeBounds=pamtra2.hydrometeors.size.logspaceBounds, #function/object to call for getting sizes
    sizeCenter = pamtra2.hydrometeors.size.boundsToMid,
    sizeBoundsWidth = pamtra2.hydrometeors.size.boundsWidth,
    sizeDistribution = pamtra2.hydrometeors.sizeDistribution.exponentialFieldWC,
    aspectRatio = 0.6,
    crossSectionArea = pamtra2.hydrometeors.crossSectionArea.powerLaw,
    mass = pamtra2.hydrometeors.mass.powerLaw,
    density = pamtra2.hydrometeors.density.softEllipsoid,
    #to do: adapt refractiveIndex routine so that mixing and RI are treated separately. 
    relativePermittivity = refractiveIndex.snow.eps,
    scattering = pamtra2.hydrometeors.scattering.Mie,
    fallVelocity = pamtra2.hydrometeors.fallVelocity.heymsfield10_particles,
    Dmin = 1e-6,
    Dmax = 1e-2,
    massSizeA = 0.0121, 
    massSizeB = 1.9,
    areaSizeA = 0.4,
    areaSizeB = 1.8,
    minDensity = 100,
    bla = 12,
    useFuncArgDefaults = True,
    )


/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2/hydrometeors/sizeDistribution.py:301: UserWarning: Truncation effect on the PSD are not considered. I.e., typically mass is lost!
  warnings.warn('Truncation effect on the PSD are not considered. '
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/dask/local.py:271: RuntimeWarning: divide by zero encountered in true_divide
  return func(*args2)


sizeBounds <function logspaceBounds at 0x11ab85510>
callable
sizeCenter <function boundsToMid at 0x11ab85598>
callable
sizeBoundsWidth <function boundsWidth at 0x11ab85620>
callable
aspectRatio 0.6
not callable 0.6
mass <function powerLaw at 0x31bb656a8>
callable
density <function softEllipsoid at 0x31bb65ea0>
callable
crossSectionArea <function powerLaw at 0x31bb65d08>
callable
sizeDistribution <function exponentialFieldWC at 0x31bb659d8>
callable
relativePermittivity <function eps at 0x41c767a60>
callable
scattering <function Mie at 0x31bb6e1e0>
callable
fallVelocity <function heymsfield10_particles at 0x31bb6e400>
callable
nBins 20
not callable 20
Dmin 1e-06
not callable 1e-06
Dmax 0.01
not callable 0.01
massSizeA 0.0121
not callable 0.0121
massSizeB 1.9
not callable 1.9
areaSizeA 0.4
not callable 0.4
areaSizeB 1.8
not callable 1.8
minDensity 100
not callable 100
bla 12
not callable 12


/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/_collections_abc.py:720: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  yield from self._mapping
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2/hydrometeors/core.py:273: UserWarning: The following kwargs were NOT used: ['bla']
  '%s' % self._keysToBeUsed)


We have access to a lot of internal properties, like the fall velocityu of particles

In [10]:
# plt.plot(
#     pam2.hydrometeors.rain.profile.sizeCenter,
#     pam2.hydrometeors.rain.profile.fallVelocity.isel(layer=10,time=0,lat=0),
#     label = 'rain')
# plt.plot(
#     pam2.hydrometeors.snow.profile.sizeCenter,
#     pam2.hydrometeors.snow.profile.fallVelocity.isel(layer=10,time=0,lat=0),
#     label = 'snow')
# plt.legend()
# plt.grid()


The profile arrays have been populated, but they contain daks arrays. These are only placeholders, they have not been calculated yet!

In [11]:
pam2.hydrometeors.rain.profile, pam2.hydrometeors.snow.profile

(<xarray.Dataset>
 Dimensions:                  (frequency: 2, lat: 10, layer: 100, sizeBin: 40, sizeBin1: 41, time: 1)
 Coordinates:
   * sizeBin                  (sizeBin) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 ...
   * sizeBin1                 (sizeBin1) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 ...
   * time                     (time) datetime64[ns] 2016-01-01
   * lat                      (lat) int64 70 71 72 73 74 75 76 77 78 79
   * layer                    (layer) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 ...
   * frequency                (frequency) float64 3.5e+10 9.4e+10
 Data variables:
     sizeBounds               (sizeBin1) float64 1e-06 0.000251 0.000501 ...
     sizeCenter               (sizeBin) float64 0.000126 0.000376 0.0006259 ...
     sizeBoundsWidth          (sizeBin) float64 0.00025 0.00025 0.00025 ...
     aspectRatio              float64 1.0
     density                  float64 1e+03
     mass                     (sizeBin) float64 1.047e-09 2.782e-08 1.284e-07 ...
     

Force computation with e.g.
`pam2.hydrometeors.rain.profile.compute()` or `pam2.hydrometeors.rain.profile.to_netcdf()` or  or `pam2.hydrometeors.rain.profile.fallVelocity.plot()`

# Radar simulator


In [12]:


pam2.addInstrument(
    pamtra2.instruments.radar.dopplerRadarPamtra,
    name = 'test',
    frequencies = 3.5e+10,    
        radarMaxV=7.885,
        radarMinV=-7.885,
        radarAliasingNyquistInterv=1,
        radarNFFT=256,
        verbosity=0,
        radarAirmotion=True,
        radarAirmotionModel="constant",  # "constant","linear","step"
        radarAirmotionVmin=0,
        radarAirmotionVmax=0,
        radarAirmotionLinearSteps=30,
        radarAirmotionStepVmin=0.5,
        radarK2=0.93,         # dielectric constant |K|² (always for liquid water by convention) for the radar equation
        radarBeamwidthDeg=0.2,
        radarIntegrationTime=60,
        radarPNoise1000=-30,
        radarNAve=150,
        seed=0,
)

> /Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2/instruments/radar.py(28)__init__()
-> super().__init__(
(Pdb) locals()
{'pdb': <module 'pdb' from '/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/pdb.py'>, 'kwargs': {'radarMaxV': 7.885, 'radarMinV': -7.885, 'radarAliasingNyquistInterv': 1, 'radarNFFT': 256, 'verbosity': 0, 'radarAirmotion': True, 'radarAirmotionModel': 'constant', 'radarAirmotionVmin': 0, 'radarAirmotionVmax': 0, 'radarAirmotionLinearSteps': 30, 'radarAirmotionStepVmin': 0.5, 'radarPNoise1000': -30, 'radarNAve': 150, 'radarBeamwidthDeg': 0.2, 'radarIntegrationTime': 60, 'seed': 0, 'momentsNPeaks': 2, 'momentsNoiseDistanceFactor': 0, 'momentsSpecNoiseMean': None, 'momentsSpecNoiseMax': None, 'momentsPeakMinSnr': -10, 'momentsPeakMinBins': 2, 'momentsSmoothSpectrum': True, 'momentsUseWiderPeak': False, 'momentsReceiverMiscalibration': 0}, 'applyAttenuation': None, 'gaseousAttenuationModel': 'Rosenkranz98', 'radarK2': 0.93, 'freque

BdbQuit: 

Again, the results are not estimated yet when using dask.

In [ ]:
pam2.instruments.test.results


Do the numbers!

In [ ]:
pam2.instruments.test.results.load()

In [ ]:
pam2.instruments.test.results.pathIntegratedAttTopDown.T.plot()

